In [1]:
import warnings; warnings.filterwarnings("ignore")
import torch
import torchvision
import tensorflow as tf
import onnx
import onnx_tf
from onnx_tf.backend import prepare

model = torchvision.models.resnet18(True)

dummy_input = torch.randn(1, 3, 224, 224) # nchw

onnx_filename = "resnet18.onnx"

torch.onnx.export(model, dummy_input,
                  onnx_filename,
                  verbose=True)

torch_output = model(dummy_input)

torch_output_np = torch_output.detach().numpy()

torch_output_np.shape

dummy_input_np = dummy_input.numpy()

dummy_input_np.shape

# step 2, create onnx_model using tensorflow as backend. check if right and export graph.
onnx_model = onnx.load(onnx_filename)
tf_rep = prepare(onnx_model, strict=False)

tf_rep.tensor_dict[tf_rep.inputs[0]], tf_rep.tensor_dict[tf_rep.outputs[0]]

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
config=tf.ConfigProto(gpu_options=gpu_options)

with tf.device("/device:cpu:0"):
    with tf_rep.graph.as_default():
        with tf.Session() as sess:
            feed_dict = {tf_rep.tensor_dict[key]: dummy_input_np for key in tf_rep.inputs}
            sess.run(tf.global_variables_initializer())
            outputs = [tf_rep.tensor_dict[output] for output in tf_rep.outputs]
            tf_output = sess.run(outputs, feed_dict=feed_dict)

In [2]:
from IPython.display import FileLink

In [3]:
FileLink('./resnet18.pb')

/home/ubuntu/on-device/pytorch/resnet18.pb

In [4]:
FileLink('./input_image.npy')

/home/ubuntu/on-device/pytorch/input_image.npy